In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings

import funcs

import pandas as pd
# import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
# from ydata_profiling import ProfileReport
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import (
    ExtraTreesRegressor,
    RandomForestRegressor,
    VotingRegressor,
)
from sklearn.impute import KNNImputer
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import MaxAbsScaler
from xgboost import XGBRegressor

%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
df = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df = df.drop(['Id'], axis=1)
df_test = df_test.drop(['Id'], axis=1)
df.rename(
    columns={
        "Strength": "target",
        "Superplasticizer": "SP",
        "Fly Ash": "FA",
        "Water": "W",
        "Coarse Aggregate": "Ag",
        "Fine Aggregate": "Af",
        "Blast Furnace Slag": "BFS",
        "Cement": "C",
        "Age": "t",
    },
    inplace=True,
)
df.info()

df_test.rename(
    columns={
        "Strength": "target",
        "Superplasticizer": "SP",
        "Fly Ash": "FA",
        "Water": "W",
        "Coarse Aggregate": "Ag",
        "Fine Aggregate": "Af",
        "Blast Furnace Slag": "BFS",
        "Cement": "C",
        "Age": "t",
    },
    inplace=True,
)
df.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C       800 non-null    float64
 1   BFS     800 non-null    float64
 2   FA      800 non-null    float64
 3   W       800 non-null    float64
 4   SP      800 non-null    float64
 5   Ag      800 non-null    float64
 6   Af      800 non-null    float64
 7   t       800 non-null    int64  
 8   target  800 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 56.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C       800 non-null    float64
 1   BFS     800 non-null    float64
 2   FA      800 non-null    float64
 3   W       800 non-null    float64
 4   SP      800 non-null    float64
 5   Ag      800 non-null    float64
 6   Af      800 non-null    float64
 

In [4]:
# from scipy.stats import zscore

# for i in df.columns:
#     df['z_score'] = zscore(df[i])
#     df = df[df['z_score'].abs() <= 3]
# df.shape

In [5]:
X = df.drop(["target"], axis=1)
y = df["target"]

X.columns

Index(['C', 'BFS', 'FA', 'W', 'SP', 'Ag', 'Af', 't'], dtype='object')

In [6]:
X_fe = funcs.engineer_features(X)

cb = CatBoostRegressor(
    iterations=1200,
    learning_rate=.9,
    depth=3,
    random_seed=137,
    eval_metric='RMSE',
    logging_level='Silent',
    loss_function='RMSE',
)

funcs.get_cv_rmse(cb, X_fe, y)


CV RMSE   : 4.56
MAPE      : 0.13


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=137
)

X_train_fe = funcs.engineer_features(X_train)
X_test_fe = funcs.engineer_features(X_test)

cb.fit(X_train_fe, y_train)
check = cb.predict(X_test_fe)
check_df = pd.DataFrame(
    y_test.reset_index(drop=True), columns=["target"]
)
check_df['pred'] = pd.DataFrame(check, columns=["pred"])
check_df

,target,pred
0,13.22,14.168811
1,21.92,19.348057
2,25.56,25.585811
3,31.74,31.908278
4,24.07,24.486467
...,...,...
75,39.29,37.049245
76,19.20,15.550771
77,33.96,43.818279
78,41.72,42.812908


In [8]:
# X_val = funcs.engineer_features(df_test)
# X_val.head()

In [9]:
# import numpy as np
# y_pred_val = cb.predict(X_test)
# y_pred_df = pd.DataFrame({'Id': range(0, len(y_pred_val)), 'Strength': np.round(y_pred_val, 1)})
# y_pred_df.to_csv('y_pred_regression.csv', index=False)